# Si446x Radio DBus Driver Access Routines

## Establish DBus connection to the Radio

In [1]:
from pydbus import SessionBus
from si446x.si446xdef import BUS_NAME, OBJECT_PATH

def si446x_dbus_open():
    #get the session bus
    bus = SessionBus()
    #get the object
    return bus.get(BUS_NAME, OBJECT_PATH)

## Get Radio Part Info

### Silicon Labs Radio Part Info

The known device version information as published on Silicon Labs Message Board:

ROMID=3,REV=B1B
4060
4063
4355
4362
4438
4455
4460
4461
4463
4464

ROMID=6,REV=C2A
4055
4060
4063
4355
4362
4438
4455
4460
4461
4463
4464

ROMID=6,REV=A2A
4467
4468

(see: http://community.silabs.com/t5/Proprietary-Knowledge-Base/Using-PART-INFO-command-to-identify-EZRadio-PRO-part-number/ta-p/194617)

In [33]:
from si446x.si446xdef import *

def si446x_dbus_part_info(radio):
    pi = []
    #create get part info radio command message
    request = read_cmd_s.parse('\x00' * read_cmd_s.sizeof())
    request.cmd='PART_INFO'
    cmd = read_cmd_s.build(request)
    #call the methods and print the results
    rsp = radio.spi_send_recv(cmd, read_part_info_rsp_s.sizeof(),
                          read_cmd_s.name, read_part_info_rsp_s.name)    
    if (rsp):
        response = read_part_info_rsp_s.parse(bytearray(rsp))
        frsp = ("Part Number: %x, rev: 0x%x, id: 0x%x, romid: 0x%x"%
              (response.part, response.chiprev, response.id, response.romid))
        pi.append((frsp, response, rsp[1:]))

    request.cmd='FUNC_INFO'
    cmd = read_cmd_s.build(request)
    rsp = radio.spi_send_recv(cmd, read_func_info_rsp_s.sizeof(),
                              read_cmd_s.name, read_func_info_rsp_s.name)    
    if (rsp):
        response = read_func_info_rsp_s.parse(bytearray(rsp))
        frsp = ("Firmware: %d.%d.%d, patch: 0x%x, func: 0x%x"%
              (response.revext, response.revbranch, response.revint, response.patch, response.func))
        pi.append((frsp, response, rsp[1:]))
    return pi


## Get GPIO Pin Configuration

In [3]:
def si446x_dbus_get_gpio_config(radio):
    request = read_cmd_s.parse('\x00' * read_cmd_s.sizeof())
    request.cmd='GPIO_PIN_CFG'
    cmd = read_cmd_s.build(request)
    rsp = bytearray(radio.spi_send_recv(cmd, get_gpio_pin_cfg_rsp_s.sizeof(),
                             read_cmd_s.name, get_gpio_pin_cfg_rsp_s.name))
    print('{}: {}'.format(get_gpio_pin_cfg_rsp_s.name, hexlify(rsp)))
    return rsp

In [4]:
def si446x_dbus_get_int_status(radio):
    request = read_cmd_s.parse('\x00' * read_cmd_s.sizeof())
    request.cmd='GPIO_PIN_CFG'
    cmd = read_cmd_s.build(request)
    rsp = bytearray(radio.spi_send_recv(cmd, get_gpio_pin_cfg_rsp_s.sizeof(),
                             read_cmd_s.name, get_gpio_pin_cfg_rsp_s.name))
    print('{}: {}'.format(get_gpio_pin_cfg_rsp_s.name, hexlify(rsp)))
    return rsp

## Get Radio Properties

Retrieve one, some, or all properties of a specific group.

If the number of properties requested (length) is greater than the Radio response message buffer size (16 bytes minus CTS), then multiple Radio get_request messages are issued and  the response is accumulated, repeating radio requests until total length has been retrieved. Note that first byte in response message is the CTS value (0xFF) and is therefore stripped.

In [5]:
from si446x.si446xdef import *
from binascii import hexlify

def si446x_dbus_get_property(radio, group, prop_x, len):
    limit = prop_x + len
    MAX_RSP = 15
    prop_b = bytearray()
    request = get_property_cmd_s.parse('\x00' * get_property_cmd_s.sizeof())
    request.cmd='GET_PROPERTY'
    request.group=group
    while (prop_x < limit):
        chunk_size = limit - prop_x
        x = MAX_RSP if (chunk_size >= MAX_RSP) else chunk_size
        request.num_props=x
        request.start_prop=prop_x
        cmd = get_property_cmd_s.build(request)
        rsp = radio.spi_send_recv(cmd, x + 1,
                             get_property_cmd_s.name, get_property_rsp_s.name)
        if (rsp):
            prop_b += bytearray(rsp[1:x+1])
            prop_x += x
        else:
            return None
    return prop_b

## Get Radio Driver Status

In [6]:
def si446x_dbus_status(radio):
    #call the methods and print the results
    return radio.status()

# Send Packets

In [7]:
from time import sleep

# buf = bytearray('\x00' * 250)
# buf[0] = 250
# pwr = 33

def si446x_dbus_send(radio, buf, pwr=32, count=1, wait=None):
    for i in range(1,count):
        buf[0] = len(buf)
        buf[1] = i
        rsp = radio.send(buf, pwr)
        if (wait): sleep(wait)
        print('.',)

# Unit Test

In [35]:
UNIT_TEST = False

In [34]:
if (UNIT_TEST):
    radio = si446x_dbus_open()
    for tup in si446x_dbus_part_info(radio):
        print(tup[0])
#        print("Part Number: %x, rev: 0x%x, id: 0x%x, romid: 0x%x"%
#              (tup[0].part, tup[0].chiprev, tup[0].id, tup[0].romid))
#        print("Firmware: %d.%d.%d, patch: 0x%x, func: 0x%x"%
#              (s[1].revext, s[1].revbranch, s[1].revint, s[1].patch, s[1].func))
    print(si446x_dbus_status(radio))
    si446x_dbus_get_gpio_config(radio)
    buf = bytearray('\x00' * 10)
    si446x_dbus_send(radio, buf)

Part Number: 4463, rev: 0x11, id: 0xf, romid: 0x3
Firmware: 3.0.15, patch: 0x0, func: 0x1
ON, <States=S_RX_ON>, unshuts 0
 RX: packets 0, len_errors 0, timeouts 0, sync_errors 0, crc_errors 0, rssi 0, 
 TX: packets 0, errors 0, timeouts 0, power 5, 
 sync frr: 08000e54
 node P211, version 0.1.53 [2017.04.30 16:31:08.645570]
get_gpio_pin_cfg_rsp_s: ff9c08a120270b00
